# Import Library

In [ ]:
import configparser
from EEG import EEG
from utils import *
import numpy as np
import mne
import matplotlib
# matplotlib.use('TKAgg')
from matplotlib import pyplot as plt
import matplotlib.transforms as mtransforms
import json
import sys
import os
import pandas as pd
from datetime import datetime
import joblib
from emg_removal_experiment import detect_emg
import random

# Input (User Edits Here)

In [ ]:
###Required
    #id of the subject. Type: int. Ex: 'id_name = 1489'
id_name = 'G07'
###Optional
    #Expected name of the ouput file. Default: 'None'
        #None: autosave output with the format: 'id.html'. Overwrite file if running same ID again
        #New:  autosave output with the format: 'id_timestamp.html'. Write a new file if running same ID again
        #Type: str: save output with the expected name. Ex: "eeg_processing_SinhNguyen"
ouput_name =''

# Initialize Variables

In [ ]:
config = configparser.ConfigParser()
config.read("config.ini")

_WINDOW = int(config["CHUNK"]["WINDOW"])
N_COMP = 10
# THRESHOLD = 0.263506042367534
THRESHOLD = 0.32729012499342763
N_SAMPLE = int(config["DEFAULT"]["N_SAMPLE_PER_SUBJECT"])
counter = 0

# Read Raw Data & Visualize

In [ ]:
#read raw data
path_edf=r"C:\Users\ASUS\Desktop\New folder\New folder\stress-detection-master\pipeline\edf\G07.edf"
path_stage= r"C:\Users\ASUS\Desktop\New folder\New folder\stress-detection-master\pipeline\edf\STAGE.csv"
eeg = EEG(path_edf=path_edf, path_stg=path_stage)
diff_time = eeg.calibration_time

In [ ]:
#plotting setting
plt.rcParams["figure.figsize"] = [10,10]

In [ ]:
!pip list

# Processing pipeline:

## Workflow: Bandpass filtering -> Baseline correction -> EOG removal -> EMG removal

### Bandpass filtering -> Baseline correction for Entire Data

In [ ]:
# Apply bandpass filter
eeg_bandpass = eeg.raw.copy()
eeg_bandpass.apply_function(butter_bandpass_filter)

# Apply baseline correction
eeg_baseline = eeg_bandpass.copy()
eeg_baseline.apply_function(baseline_calc)
print(eeg.raw)
print(eeg_baseline)

### Get all Tasks after Bandpass + Baseline

In [ ]:
def get_data(task,processed_eeg):
    _min, _max = np.min(task)+ diff_time, np.max(task) + diff_time

    # Get raw data
    sample_raw = processed_eeg.copy().crop(_min, _max, include_tmax=False)
    
    return sample_raw.get_data().T

#Concatenate data of all tasks
task_0=get_data(eeg._tasks[0],eeg_baseline)
task_1=get_data(eeg._tasks[1],eeg_baseline)
task_2=get_data(eeg._tasks[2],eeg_baseline)
task_data = np.concatenate((task_0, task_1, task_2), axis=0)
print(task_data.shape)

### Get random task chunk

In [ ]:
##Use a.Tho's Coding
def get_random_task_chunk(eeg,processed_eeg, window):
    # Getting random chunk of data from raw signal
    _chunk = random.choice(list(chunks(random.choice(eeg.tasks), window)))
#     _chunk = random.choice(list(chunks(eeg.tasks[1], window)))
    _min, _max = np.min(_chunk) + diff_time, np.max(_chunk) + diff_time

    # Preprocessing
    # Getting sample data
#     sample_raw = eeg.raw.copy().crop(_min, _max, include_tmax=False).get_data().T
    sample_raw = processed_eeg.copy().crop(_min, _max, include_tmax=False)

    return sample_raw

### Load classification models (eog & emg)

In [ ]:
###Load classification models
    #load eog model
clf_eog = joblib.load(r"C:\Users\ASUS\Desktop\New folder\New folder\stress-detection-master\pipeline\model\emg_classifier_20200814.joblib")
    #load emg model
clf_emg = joblib.load(r"C:\Users\ASUS\Desktop\New folder\New folder\stress-detection-master\pipeline\model\eog_classifier_v2.joblib")

### Get EOG chunks

In [ ]:
eeg_remove_eog = eeg_baseline.copy()

### Get EMG chunks

### Process for each task of the subject, plot for each chunk (default 15s) of each task

In [ ]:
i=0
for task in eeg._tasks:
    i+=1
    for idx, chunk in enumerate(chunks(task, _WINDOW)):

        if counter > N_SAMPLE:
            break

        _min, _max = np.min(chunk), np.max(chunk)

        # Get raw data
        sample_raw = eeg.raw.copy().crop(_min, _max, include_tmax=False)
        #Plot raw
        raw = sample_raw.get_data().T
        plot_raw(raw, eeg, title=f"Task{i}_Chunk{idx}_Raw")
        
        # Apply bandpass filter
        sample_raw_bandpass = sample_raw.copy()
        sample_raw_bandpass.apply_function(butter_bandpass_filter)

        # Apply baseline correction
        sample_raw_baseline = sample_raw_bandpass.copy()
        sample_raw_baseline.apply_function(baseline_calc)

        #Plot After Bandpass + Baseline
        baseline = sample_raw_baseline.get_data().T
        plot_raw(baseline, eeg, title=f"Task{i}_Chunk{idx}_BandPass_Baseline")
        
        #Detect EOG
        ica, sample_raw_removed_eog = ica_pipe(sample_raw_baseline, clf_eog)
        #Plot After Detecting EOG
        eog_removal = sample_raw_removed_eog.get_data().T
        plot_components(c1, N_COMP,'ICA comp')
        plot_raw(eog_removal, eeg, title=f"Task{i}_Chunk{idx}_EOG_Removal")
        plot_psd(raw, eog_removal, eeg, figsize=(25, 60), title="PSD plots after EOG removal")
        
        # Detect EMG
        cca, c1, list_of_emg, list_of_probs = detect_emg(eog_removal, clf_emg, N_COMP)
        if list_of_emg is not None and len(list_of_emg) > 0:
            # Plot CCA components
            plot_components(c1, N_COMP)
            # Zeroing out the chosen components
            print("Found EMG in the following components: {}".format(str(list_of_emg).strip('[]')))
            print("Corresponding probabilities: {}".format(str(list_of_probs).strip('[]')))
            # print(list_of_probs)
            c1[:, list_of_emg] = 0
            corrected = cca.inverse_transform(c1)

            # Plot the results
            plot_raw(corrected, eeg, title=f"Task{i}_Chunk{idx}_EMG-removed")
            plot_psd(raw, corrected, eeg, figsize=(25, 60), title="PSD plots after EMG removal")

            # plot_psd(corrected, eeg, figsize=(25, 60), title="PSD - EMG-removed")
        else:
            # Plot CCA components
            plot_components(c1, N_COMP)
            print(f"Task{i}_Chunk{idx}_No EMG Component to Remove")


# Export .html file in the folder "html"

In [ ]:
# import os
# command = 'jupyter nbconvert preprocessing_workflow.ipynb --to html --output ./html/'
# if ouput_name == None:
#     os.system(command+ f"{id_name}.html")
# elif ouput_name == "New":
#     from datetime import datetime
#     now = datetime.now()
#     timestamp = now.strftime("%Y%m%d_%H%M%S")
# #     os.system(command + str(id_name)+'_'+ str(timestamp)+'.html')
#     os.system(command + f"{id_name}_{timestamp}.html")
# else:
#     os.system(command + f"{ouput_name}.html")